# Intelligent Systems 2020: 3rd  practical assignment 
## Games and Adversarial Search 
### An introduction to the Schnapsen platform

Your name: Mark Melnic

Your VUnetID: mmc560

If you do not provide your name and VUnetID we will not accept your submission. 

### Learning objectives
At the end of this exercise you should be able to use the Schnapsen platform, run basic games between agents, and run tournaments in order to evaluate rational agents (also called bots). You should also be able to identify the working patterns of the MiniMiax algorithm in this platform and the improvements with Alpha/Beta pruning. 

1. Understand the main functionality of the Schnapsen platform (playing games and running tournements)
2. Implement your own rational agents (bots)
2. Follow the individual steps and explain the MiniMax algorithm
3. Make small modifications of the code to see the effect on the search algorithms
4. Make small adaptations to the algorithm to study the computational properties

### Practicalities

Follow this Notebook step-by-step. 

Of course, you can do the exercises in any Programming Editor of your liking. But you do not have to. Feel free to simply write code in the Notebook. Please use your studentID+Assignment3.ipynb as the name of the Notebook.  

Note: unlike the courses dedicated to programming we will not evaluate the style of the programs. But we will, however, test your programs on other data that we provide, and your program should give the correct output to the test-data as well.

As was mentioned, the assignment is graded as pass/fail. To pass you need to have either a full working code or an explanation of what you tried and what didn't work for the tasks that you were unable to complete (you can use multi-line comments or a text cell).

## Initialising 

First, let us make sure the necessary packages are installed, and imported. Run the following code:

In [ ]:
import sys, random

from api import State, engine, util

## Playing the first games

The basic engine comes with three basic bots: rand, bully and rdeep (the rest you can ignore for now). To try them out, just run the following bit of code. 

In [ ]:
# Choose your first player
player1 = "rand"
player2 = "bully"
# Decide in which phase you want to start the game. 
startphase = 1
# Decide whether you want verbose output or not 
verbose=True 

#And here you run a game on the engine. 
engine.play(util.load_player(player1),util.load_player(player2), state=State.generate(phase=startphase), max_time=10000, verbose=verbose)

Running engine.play provides some textual output to show how the game is progressing. At every plie (half a turn) you will see what move the player made and a concise overview of the board. Something like this (when you run a game in verbose mode):

> Player 1 plays: KC
> The game is in phase: 1<br>
> Player 1's points: 21, pending: 0<br>
> Player 2's points: 25, pending: 0<br>
> Player 1's hand: 10C JC AD QD QH<br>
> Player 2's hand: AC KD 10H KS QS<br>
There are 2 cards in the stock<br>


The first line signifies that player 1 has played the King of Clubs card. Internally these cards are represented by indices from 0 to 19. To make the translation between indices and card names, use util.get_card_name(index), which returns the rank and the suit of the card as a tuple. Alternatively, use util.get_rank(index) and util.get_suit(index) for each property alone. In this case it is a King of Clubs. Have a look at the GitHub readme or at the top of __deck.py to see the convention used for encoding cards into indices.

In [ ]:
util.get_card_name(2)


You can also run the Python programmes provided from the command line, or in your favourite editor. <br>
Run 
> python play.py -1 rand -2 bully 

to run the rand bot against the bully bot, or 
> python play.py -h 

to see other options. 




There is a lot of randomness involved in the game when the cards are distributed to the players and the pile. To get an accurate sense of whether one player is better than another, you'll need to play a number of different games. The following code will play a tournament between bully and rand where every pair of participants plays 10 matches. 


In [ ]:
botnames = []
verbose = False 
myphase = 1
myrepeats = 10

# Create player 1
player1 = util.load_player("rand") 
player2 = util.load_player("bully") 

bots = [player1,player2]

n = len(bots)
wins = [0] * len(bots)
matches = [(p1, p2) for p1 in range(n) for p2 in range(n) if p1 < p2]

totalgames = (n*n - n)/2 * myrepeats
playedgames = 0

print('Playing {} games:'.format(int(totalgames)))
for a, b in matches:
    for r in range(myrepeats):

        if random.choice([True, False]):
            p = [a, b]
        else:
            p = [b, a]

        # Generate a state with a random seed
        state = State.generate(phase=myphase)

        winner, score = engine.play(bots[p[0]], bots[p[1]], state, 1000, verbose, True)

        if winner is not None:
            winner = p[winner - 1]
            wins[winner] += score

        playedgames += 1
        print('Played {} out of {:.0f} games ({:.0f}%): {} \r'.format(playedgames, totalgames, playedgames/float(totalgames) * 100, wins))

print('Results:')
for i in range(len(bots)):
    print('    bot {}: {} points'.format(bots[i], wins[i]))


### Task 1: 
The previous code runs a tournament between rand and bully, but you can adapt the script by testing the performance of these bots with the third default bot, rdeep, as the opponent. The general idea of rdeep was extensively discussed under the header PIMS (Perfect Information Monte Carlo Sampling). Report in the following Markdown Cell on the results you get from two-player tournaments including rdeep, rand and bully (rdeep vs. rand; rdeep vs. bully). Describe which games you played, and who won. Add the (non-verbose) output of the script.


### Task 2: 
The previous code runs a tournament between two bots only, but you can easily adapt the script above to play round-robin tournament. All you have to do is to add a third player to the bots list. Report in the following Markdown Cell on the results you get from three-player tournament including rdeep, rand and bully. Add the (non-verbose) output of the script. Report on the results of the tournament and try to explain in your own words what do the results mean.


## Inspecting the code

Now let's have a look at how the bots work. Open the file bots/rand/rand.py in PyCharm. Each bot is a class called Bot. 

> We will use more advanced features of Python than what you have seen so far in Introduction to Python (don’t worry), so for more details have a look at:
>    https://www.learnpython.org/en/Classes_and_Objects

The rand bot contains nothing but an empty constructor, and one method: get_move(self, state). This is the only method you need to implement to get a working bot. It receives a description of a game state, and returns a move. A move is always a pair of two elements, each of which can be either an integer or None. Note that it is not an option to pass, therefore (None, None) is not a valid move.

As you can see, in the rand bot, the state object does almost all the work: state.moves() gives you a list of legal moves. The rand Bot simply makes a random choice from this list using the function random.choice() from the python library.

If you want to see what happens when you make a given move, just do
next_state = state.next(move)
And you get a state representing the outcome.

What else can the State object do for you? You can look at the code in api/_state.py.

### bully.py

Bully is a deterministic bot: given the same state it will always do the same thing. We've removed part of the explanation from the comments. 

### Task 3: 
Have a look at the code: describe in your own words what strategy does the bully bot use? 

### rdeep.py
The lectures discuss the hill-climbing strategy: look one move ahead and pick the move that leads to the best heuristic. The heuristics we use is the ratio of the player points w.r.t. to the total points currently assigned in the game. The higher this value, the better the state is for us. Imagine doing hill-climbing with this heuristic. This strategy would not work here. Why not? 



In order to avoid this issue, we need to loook further ahead than the hill climbing strategy does. rdeep.py does this in the simplest way we could think of. Make eight random moves and look at the value of the resulting state. Do this a few times and average the values found. This method is called Perfect-Information Monte-Carlo Sampling (PIMC).

You just ran a tournament between rdeep and the other two bots. Most likely, rdeep will have won a few more games. But does the difference really mean rdeep is better? It might just be that rdeep is no better than rand and won by pure luck.

### Task 4 
If you wanted to provide scientific evidence that rdeep is better than rand, how would you go about it?

### mybot.py

It's time to write your own bot. Think of a simple strategy that is easy to implement. To create the bot follow these steps:
1. Create the directory bots/mybot (in the directory, not this notebook!)
2. Add an empty file __init__.py, or copy it from one of the other bot directories.
3. Copy rand.py to the directory mybot, and rename it to mybot.py
4. Change the implementation of get_move(state). Keep the method signature (line 16) exactly as it is.

Make sure your bot always returns either a pair elements that are each either int or None. Try playing a tournament against rand. See if you can get a decent margin.

If your bot has parameters (like a search depth, or a pre-programmed probability of doing nothing) you can add these to the constructor. Have a look at rdeep.py to see how this is done.

To get some examples of how to talk to the API, see the README.md

### Task 5 
Add your implementation of get_move() and the result of a tournament against rand to your report. 

Please write your code here (in raw text, to avoid an error), as well as the results in the following cell: 


## MINIMAX Adversarial Games
Now we will build some bots that search the game tree by using the MiniMax algorithm and show genuinely intelligent behavior. Because we only have partial information, and as there would be too many belief-states, we play these bots on the 2nd phase of the game only, when the stock has been exhausted, such that the state of all cards is known and no assumptions have to be made.

All you need to do to finish the minimax bot is to add one line of code on line 58. The skeleton given in the code differs slightly from the pseudocode from the lecture in that instead of having two sub-methods, all is implemented in one method. Take your time to really understand the minimax algorithm, recursion, and the rest of the code. The pseudocode would then look like as follows 

### Task 6
Implement the parts of the algorithm marked "???" in the code of the minimax.py program in the bot directory. Please write down your new code in the following raw cell. 

Once you have done this, you can check whether you algorithm runs correctly.

In [ ]:
# Choose your first player
player1 = "rand"
player2 = "minimax"
# Decide in which phase you want to start the game. 
startphase = 2
# Decide whether you want verbose output or not 
verbose=True 

#And here you run a game on the engine. 
engine.play(util.load_player(player1),util.load_player(player2), state=state.generate(phase=startphase), max_time=10000, verbose=verbose)

## Heuristics 

What heuristic do these implementations use? Explain the heuristic function in your own words: 

Adapt the search depth (in the constructor of the Minimax bot, and adapt the Minimax procedure by uncommenting the respective function) to smaller values (search depths). Report on the performance of these different versions of using the heuristics against some other players (like rdeep and rand), and see if the performance (nr. of games won) differs with the different look-aheads. 

### Task 7
Report on your experiments with Minimax as compared to previous bots (or a copy of MiniMax with a different search depths)

## Alphabeta

Finally let us look at Alphabeta pruning. Alphabeta pruning is a technique to make minimax faster. If implemented correctly, it will do exactly the same  thing minimax does, but skip large parts of the search tree. We've provided a basic implementation in bots/alphabeta/alphabeta.py. 

### Task 8

Once again, one crucial bit of the implementation are missing, the decision on when to prune. Finish the implementation of the alphabeta bot. Copy the line of code you adapted in the skeleton file alphabeta.py into the following cell: 



The following programme lets you see if you implemented alphabeta and minimax correctly. Run it, and check the outcome. 


In [ ]:
from api import State, util
import random, time

from bots.alphabeta import alphabeta
from bots.minimax import minimax

REPEATS = 100
DEPTH = 6

ab = alphabeta.Bot(randomize=False, depth=DEPTH)
mm = minimax.Bot(randomize=False, depth=DEPTH)

mm_time = 0
ab_time = 0

# Repeat
for r in range(REPEATS):
    
    # Repeat some more 
    for r2 in range(REPEATS):

        # Generate a starting state
        state = State.generate(phase=2)

        # Ask both bots their move
        # (and time their responses)

        start = time.time()
        mm_move = mm.get_move(state)
        mm_time += (time.time() - start)

        start = time.time()
        ab_move = ab.get_move(state)
        ab_time += (time.time() - start)


        if mm_move != ab_move:
            print('Difference of opinion! Minimax said: {}, alphabeta said: {}. State: {}'.format(mm_move, ab_move, state))
        else:
            print('Agreed.')

print('Done. time Minimax: {}, time Alphabeta: {}.'.format(mm_time/REPEATS, ab_time/REPEATS))
print('Alphabeta speedup: {} '.format(mm_time/ab_time))

